# Основы программирования в Python 

*Алла Тамбовцева*

## Практикум 13. Парсинг с `BeautifulSoup`: тэги и атрибуты

Мы уже немного познакомились со структурой html-страниц и поиском информации по тэгам, теперь попробуем выгрузить информацию из более интересной страницы, а точнее, с сайта [nplus1.ru](https://nplus1.ru/). Наша задача – выгрузить недавние новости в датафрейм `pandas`, чтобы потом сохранить все в файл Excel.

Для работы нам снова понадобится модуль `requests` для отправки запросов, для «подключения» к странице и получения ее содержимого в виде строки, и функция `BeautifulSoup` из библиотеки `bs4` для удобного поиска по полученной строке:

In [1]:
import requests
from bs4 import BeautifulSoup

Сохраним ссылку на главную страницу в переменную `main` и отправим запрос к ней с помощью функции `get()` из `requests`:

In [2]:
main = "https://nplus1.ru/"
page = requests.get(main)

Заберём исходный код страницы и преобразуем строку с ним в объект `BeautifulSoup`:

In [3]:
soup = BeautifulSoup(page.text)

Чтобы сгрузить все новости с главной страницы сайта, нужно собрать все ссылки на страницы с этими новостями. Ссылки в html-файле всегда заключены в тэг `<a></a>` и имеют атрибут `href`. Найдем кусочки кода HTML, соответствующие всем ссылкам на главной странице сайта:

In [4]:
links_raw = soup.find_all("a") 
links_raw[10:20]  # несколько штук для примера

[<a class="hover:underline transition-colors duration-75" href="/search/empty/768">Генетика</a>,
 <a class="hover:underline transition-colors duration-75" href="/search/empty/890">Математика</a>,
 <a class="hover:underline transition-colors duration-75" href="/search/empty/871">Космонавтика</a>,
 <a class="hover:underline transition-colors duration-75" href="/search/empty/876">Археология</a>,
 <a class="hover:underline transition-colors duration-75" href="/search/empty/775">Нейронауки</a>,
 <a class="hover:underline transition-colors duration-75" href="/search/empty/767">На мышах</a>,
 <a class="hover:underline transition-colors duration-75" href="/search/empty/771">Звук</a>,
 <a class="hover:underline transition-colors duration-75" href="/search/empty/772">Красота</a>,
 <a class="hover:underline transition-colors duration-75" href="/search/empty/778">Научные закрытия</a>,
 <a class="hover:underline transition-colors duration-75" href="/search/empty/917">ИИ спешит на помощь</a>]

Каждый элемент возвращаемого списка имеет тип `BeautifulSoup` и структуру, очень похожую на словарь. Например, ссылка `<a class="hover:underline transition-colors duration-75" href="/search/empty/768">Генетика</a>` изнутри выглядит как словарь следующего вида:

    {'href' : '/search/empty/768', 
     'class' : 'hover:underline transition-colors duration-75'}.
    
Как мы помним, значение по ключу из словаря можно вызвать с помощью метода `.get()`. Давайте извлечем значения по ключу `href` из каждого элемента списка `links`:

In [5]:
links = [li.get("href") for li in links_raw] 
links[10:20]  # несколько штук для примера

['/search/empty/768',
 '/search/empty/890',
 '/search/empty/871',
 '/search/empty/876',
 '/search/empty/775',
 '/search/empty/767',
 '/search/empty/771',
 '/search/empty/772',
 '/search/empty/778',
 '/search/empty/917']

Ссылок в списке выше много. Но нам нужны только новости – ссылки, которые начинаются с `https://nplus1.ru/news`. Создадим пустой список `news` и будем добавлять в него только ссылки, которые удовлетворяют этому условию.

In [6]:
news = []
for li in links:
    if "https://nplus1.ru/news/" in li:
        news.append(li)

In [7]:
news[0:10]

['https://nplus1.ru/news/2015/09/21/editor-thy-name',
 'https://nplus1.ru/news/2023/05/19/octopus-nightmare',
 'https://nplus1.ru/news/2023/05/20/blue-origin-win-artemis',
 'https://nplus1.ru/news/2023/05/20/inscultura',
 'https://nplus1.ru/news/2023/05/20/neanderthal-bone-industry',
 'https://nplus1.ru/news/2023/05/19/permafrost',
 'https://nplus1.ru/news/2023/05/19/ancient-hearth',
 'https://nplus1.ru/news/2023/05/19/x-ray-forgery-update',
 'https://nplus1.ru/news/2023/05/19/hoolock-tianxing',
 'https://nplus1.ru/news/2023/05/19/sn-2020-eyj']

Первая ссылка ведет не совсем на новость, скорее, на объявление, поэтому давайте ее уберем:

In [8]:
news = news[1:] 

Теперь наша задача сводится к следующему: изучить одну страницу с новостью, научиться из нее вытаскивать текст и всю необходимую информацию, а потом применить весь набор действий к каждой ссылке из `news` в цикле. Посмотрим на новость с индексом 0, у вас может быть другая, новости обновляются.

In [9]:
link0 = news[0]
print(link0)

https://nplus1.ru/news/2023/05/19/octopus-nightmare


### Задача 1

Отправьте запрос к странице по ссылке `link0` с одной новостью, получите результат в виде объекта `BeautifulSoup` и сохраните его как `soup0`.

In [10]:
page0 = requests.get(link0)
soup0 = BeautifulSoup(page0.text)

### Задача 2

Найдите заголовок новости и сохраните его в переменную `title`.

In [11]:
title = soup0.find("title").text

### Задача 3

Найдите имя автора новости и дату её публикации. Сохраните их в `author` и `date` соответственно.

In [12]:
date = soup0.find("meta", {"itemprop" : "datePublished"}).get("content") 
author = soup0.find("meta", {"name" : "author"}).get("content") 
print(date, author)

2023-05-19 Сергей Коленов


### Задача 4

Найдите сложность новости и рубрики, к которым она относится. Сохраните сложность в переменную `diffc`. Рубрики сначала можно сохранить в список, а затем его элементы объединить в одну строку `rubs`.

**Подсказка:** чтобы упростить себе жизнь, найдите сначала раздел (`div`), в котором на странице хранится дата и время публикации, сложность и рубрики, а затем выполняйте поиск в рамках этого раздела.

In [13]:
# начнем в следующий раз отсюда

div = soup0.find("div", 
                 {"class" : "flex flex-wrap lg:mb-10 gap-2 text-tags xl:pr-9"})
diffc = div.find_all("span")[3].text
rubs_raw = div.find_all("span")[4:] 
rubs_L = [r.text for r in rubs_raw] 
rubs = ", ".join(rubs_L)
print(rubs)

Зоология


### Задача 5

Соберите из абзацев текст новости и сохраните его в переменную `text`. Избавьтесь от постронних символов (`\xa0`, `\n`) в тексте. 

In [ ]:
### YOUR CODE HERE ###

### Задача 6

Напишите функцию `get_news()`, которая принимает на вход ссылку на страницу с одной новостью, а возвращает список из следующих характеристик: имя автора, дата публикации, сложность новости, рубрики, текст новости.

In [ ]:
### YOUR CODE HERE ###

Отлично! Теперь осталось применить функцию ко всем ссылкам в списке `news`. Чтобы сайт не понял, что мы его автоматически грабим, будем выгружать новости постепенно – с задержкой в 1.5 секунды. Импортируем для этого функцию `sleep` :

In [ ]:
from time import sleep

Теперь будем применять функцию в цикле к каждой ссылке в `news`, только с одним дополнением – добавленной конструкцией `try-except`, которая позволит продолжать исполнение цикла, если при применении функции Python столкнулся с ошибкой любого вида:

In [ ]:
info = []
for n in news:
    # пробуй исполнить следующий код
    try:
        res = get_news(n)
        info.append(res)
        print(n)
    # если он вызвал ошибку, печатай сообщение и иди дальше
    except:
        print("Something went wrong")
        print(n)
    sleep(1.5)

Посмотрим на несколько элементов `info`:

In [ ]:
info[10:12]

Финальный штрих – импортируем `pandas` и преобразуемый полученный список кортежей в датафрейм:

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(info)
df.head()

Добавим содержательные названия столбцов и выгрузим датафрейм в файл:

In [ ]:
df.columns = ["title", "author", "date", "diffc", "rubrics", "text"]
df.to_excel("nplus1.xlsx")